In [2]:
import json
import pandas as pd

In [4]:
data = pd.read_csv('analysis/data/draft_order.csv')

In [5]:
# drop the first column
data = data.drop(data.columns[0], axis=1)
data.head(5)

,draftRank,eligibleSlots,playerId,name,projPoints
0,1,"[11, 12, 13, 14, 16, 17]",39832,Shohei Ohtani,588.0
1,2,"[10, 5, 12, 16, 17]",36969,Juan Soto,602.0
2,3,"[13, 14, 16, 17]",32081,Gerrit Cole,629.0
3,4,"[3, 7, 19, 11, 12, 16, 17]",32801,Jose Ramirez,676.0
4,5,"[13, 14, 16, 17]",39878,Corbin Burnes,613.0


In [15]:
# We will extend the draft data with a one-hot encoding of the player's eligible positions
posEligData = pd.DataFrame()
for row in data.iterrows():
  newRow = pd.Series(
    {
      **row[1],
      "c": True if '0' in row[1][1] else False,
      "1b": True if '1' in row[1][1] else False,
      "2b": True if '2' in row[1][1] else False,
      "3b": True if '3' in row[1][1] else False,
      "ss": True if '4' in row[1][1] else False,
      "of": True if '5' in row[1][1] else False,
      "dh": True if '11' in row[1][1] else False,
      "sp": True if '14' in row[1][1] else False,
      "rp": True if '15' in row[1][1] else False,
    }
  )

  posEligData = pd.concat([posEligData, newRow], axis=1)

posEligData = posEligData.T
posEligData = posEligData.reset_index(drop=True)
posEligData.head(5)

,draftRank,eligibleSlots,playerId,name,projPoints,c,1b,2b,3b,ss,of,dh,sp,rp
0,1,"[11, 12, 13, 14, 16, 17]",39832,Shohei Ohtani,588.0,False,True,True,True,True,False,True,True,False
1,2,"[10, 5, 12, 16, 17]",36969,Juan Soto,602.0,True,True,True,False,False,True,False,False,False
2,3,"[13, 14, 16, 17]",32081,Gerrit Cole,629.0,False,True,False,True,True,False,False,True,False
3,4,"[3, 7, 19, 11, 12, 16, 17]",32801,Jose Ramirez,676.0,False,True,True,True,False,False,True,False,False
4,5,"[13, 14, 16, 17]",39878,Corbin Burnes,613.0,False,True,False,True,True,False,False,True,False


In [12]:
# Assemble a list that contains the order of each pick in the draft
draft_order_list = []

# We will use the draft-order.json file which describes the order that Fantasy teams will pick in the draft
with open('analysis/data/draft-order-2023.json', 'r') as f:
  draft_order = json.load(f)
  draft_order = sorted(draft_order, key=lambda k: k['draftOrder'])
  rounds = 27

  # We will loop through the draft order and add the team name to the list for each pick
  # There will be some conditions applied to account for the "snake" draft order pattern
  for i in range(rounds):
    reverse_toggle = i % 2

    if i < 3:
      reverse_toggle = 1

    if reverse_toggle == 1:
      for player in draft_order:
        draft_order_list.append(player['teamName'])
    elif reverse_toggle == 0:
      for player in draft_order[::-1]:
        draft_order_list.append(player['teamName'])

print("There will be", len(draft_order_list), "picks over", int(len(draft_order_list) / len(draft_order)), "rounds")

There will be 324 picks over 27 rounds


In [31]:
predicted_draft_picks = [{'teamName': team['teamName'], 'picks': []} for team in draft_order]

In [ ]:
def generatePick(picker, pick_number, current_pick_list, remaining_picks, full_draft_board):
  owners_current_picks = [pick['picks'] for pick in current_pick_list if pick['teamName'] == picker]
  position_starters_config = {"c": 1, "1b": 1, "2b": 1, "3b": 1, "ss": 1, "of": 3, "dh": 1, "sp": 6, "rp": 3, "UTIL": 4, "BENCH": 5}
  # we are going to generate a "need" score for each position based on how many players of that position have already been drafted by the team owner
  pos_scores = {"c": 1, "1b": 1, "2b": 1, "3b": 1, "ss": 1, "of": 1, "dh": 1, "sp": 1, "rp": 1}
  for pick in owners_current_picks:
    player = full_draft_board.loc[full_draft_board['playerId'] == pick[0]['playerId']]
    players_pos = [index for index, pos in player[['c', '1b', '2b', '3b', 'ss', 'of', 'dh', 'sp', 'rp']].items() if pos == True]
    for pos in players_pos:
      if pos in ['c', '1b', '2b', '3b', 'ss', 'of', 'dh']:
        pos_scores[pos] -= lambda x: sum({k: v for k, v in position_starters_config.items() if k in [pos, 'UTIL', 'BENCH']}.values()) / sum({k: v for k, v in position_starters_config.items() if k in ['c', '1b', '2b', '3b', 'ss', 'of', 'dh', 'UTIL', 'BENCH']}.values())
      elif pos in ['sp', 'rp']:
        pos_scores[pos] -= lambda x: sum({k: v for k, v in position_starters_config.items() if k in [pos, 'BENCH']}.values()) / sum({k: v for k, v in position_starters_config.items() if k in ['sp', 'rp', 'BENCH']}.values())

In [136]:
pick_number = 1
for picker in draft_order_list:
  for team in predicted_draft_picks:
    if team['teamName'] == picker:
      pick_index = generatePick(picker, pick_number, predicted_draft_picks, posEligData, data)
      team['picks'].append(posEligData.iloc[pick_index][['playerId', 'name']].to_dict())
      posEligData = posEligData.drop(pick_index)
      pick_number += 1


NameError: name 'generatePick' is not defined

In [147]:
position_starters_config = {"c": 1, "1b": 1, "2b": 1, "3b": 1, "ss": 1, "of": 3, "dh": 1, "sp": 6, "rp": 3, "UTIL": 4, "BENCH": 5}
pos_scores = {"c": 1, "1b": 1, "2b": 1, "3b": 1, "ss": 1, "of": 1, "dh": 1, "sp": 1, "rp": 1}

In [154]:
pos = 'of'
if pos in ['c', '1b', '2b', '3b', 'ss', 'of', 'dh']:
  pos_scores[pos] -= (1 / 27) * sum({k: v for k, v in position_starters_config.items() if k in [pos, 'UTIL', 'BENCH']}.values())
elif pos in ['sp', 'rp']:
  pos_scores[pos] -= (1 / 27) * sum({k: v for k, v in position_starters_config.items() if k in [pos, 'BENCH']}.values())

In [155]:
pos_scores

{'c': -0.11111111111111105,
 '1b': 1,
 '2b': 1,
 '3b': 1,
 'ss': 1,
 'of': 0.5555555555555556,
 'dh': 1,
 'sp': 1,
 'rp': 1}